# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

Harmonic Mean of F and A: 0.25
F1: 0.17
Claim Acc:0.43

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
# Monte the files from MyDrive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# install packages for hugging face
!pip install sentence-transformers faiss-cpu tqdm nltk datasets lemminflect spacy



^C


   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/15.0 MB 9.2 MB/s eta 0:00:02
   ------------- -------------------------- 5.0/15.0 MB 13.1 MB/s eta 0:00:01
   --------------------- ------------------ 8.1/15.0 MB 14.0 MB/s eta 0:00:01
   --------------------------- ------------ 10.5/15.0 MB 13.4 MB/s eta 0:00:01
   -------------------------------- ------- 12.3/15.0 MB 12.3 MB/s eta 0:00:01
   ------------------------------------- -- 14.2/15.0 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/769.7 kB ? eta -:--:--
   ---------------------------------------- 769.7/769.7 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
    --------------------------------------- 3.1/212.5 MB 16.8 MB/s eta 0:00:13
   - -------------------------------------- 6.6/212.5 MB 16.8 MB/s eta 0:00:13
 

In [1]:
# Import all the packages
import numpy as np
import json
from collections import Counter
from sentence_transformers import SentenceTransformer
import pickle
import faiss
from tqdm import tqdm
import nltk
import re
from datasets import Dataset
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
from transformers import BertTokenizer
from torch.utils.data import Dataset
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

label2id = {
    "SUPPORTS": 0,
    "REFUTES": 1,
    "NOT_ENOUGH_INFO": 2,
    "DISPUTED": 3
}

class ClaimEvidenceDataset(Dataset):
    def __init__(self, claims, evidence_dict, tokenizer, max_length=512):
        self.encodings = []
        self.labels = []
        for claim_data in claims.values():
            claim_text = claim_data["claim_text"]
            label_str = claim_data["claim_label"]
            for eid in claim_data.get("evidences", []):
                if eid in evidence_dict:
                    evidence_text = evidence_dict[eid]
                    encoded = tokenizer(
                        claim_text,
                        evidence_text,
                        padding="max_length",
                        truncation=True,
                        max_length=max_length,
                        return_tensors="pt"
                    )
                    self.encodings.append({k: v.squeeze() for k, v in encoded.items()})
                    self.labels.append(label2id[label_str])

    def __getitem__(self, idx):
        item = self.encodings[idx]
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [1]:
import json, random
from collections import defaultdict, Counter
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
PAD, MASK = tokenizer.pad_token_id, tokenizer.mask_token_id

label2id = {"SUPPORTS": 0, "REFUTES": 1, "NOT_ENOUGH_INFO": 2, "DISPUTED": 3}
id2label = {v: k for k, v in label2id.items()}


# =========  数据集 ========= #
class ClaimEvidenceDataset(Dataset):
    """
    - balance=True  : 对所有标签补到 max_count * target_ratio
    - augmenters    : 可选 {'dropout','swap','pad','cutmix'}
    - aug_params    : 各方法的细粒度超参
    """
    def __init__(self,
                 claims: dict,
                 evidence_dict: dict,
                 tokenizer,
                 max_length: int = 512,
                 balance: bool = True,
                 target_ratio: float = 1.0,
                 augmenters=None,
                 aug_params=None,
                 seed: int = 42):

        random.seed(seed)
        self.tokenizer, self.max_length = tokenizer, max_length
        self.encodings, self.labels = [], []
        self.augmenters = set(augmenters or ['dropout', 'swap', 'pad', 'cutmix'])

        # 默认超参
        _default = dict(dropout_prob=0.15,
                        swap_prob=0.10,
                        pad_prob=0.05,
                        cutmix_min=0.3,
                        cutmix_max=0.7)
        self.aug_params = {**_default, **(aug_params or {})}

        # --------- 原始样本编码 --------- #
        for cdict in claims.values():
            claim_text = cdict["claim_text"]
            lab = label2id[cdict["claim_label"]]
            for eid in cdict.get("evidences", []):
                if eid in evidence_dict:
                    evi = evidence_dict[eid]
                    toks = tokenizer(claim_text, evi,
                                     truncation=True,
                                     padding="max_length",
                                     max_length=max_length,
                                     return_tensors="pt")
                    self.encodings.append({k: v.squeeze(0) for k, v in toks.items()})
                    self.labels.append(lab)

        # --------- Oversample with online augmentation --------- #
        if balance:
            self._balance_dataset(target_ratio)

    # ======= Dataset API ======= #
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v.clone() for k, v in self.encodings[idx].items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    # ======= 不同标签补样本 ======= #
    def _balance_dataset(self, target_ratio: float):
        by_label = defaultdict(list)
        for i, y in enumerate(self.labels):
            by_label[y].append(i)

        max_count = int(max(len(v) for v in by_label.values()) * target_ratio)

        for lab, idx_list in by_label.items():
            need = max_count - len(idx_list)
            for _ in range(max(0, need)):
                base_idx = random.choice(idx_list)
                base_enc = self.encodings[base_idx]
                aug_enc = self._augment_encoding(base_enc, lab)
                self.encodings.append(aug_enc)
                self.labels.append(lab)

    # ======= 单条样本增广 ======= #
    def _augment_encoding(self, enc, lab):
        enc = {k: v.clone() for k, v in enc.items()}  # 不要改原 tensor
        choice = random.choice(list(self.augmenters))
        if choice == 'dropout':
            self._token_dropout(enc)
        elif choice == 'swap':
            self._swap_neighbor(enc)
        elif choice == 'pad':
            self._random_pad(enc)
        elif choice == 'cutmix':
            self._cutmix(enc, lab)
        return enc

    # ----- 1. Random token dropout -----
    def _token_dropout(self, enc):
        ids = enc['input_ids']
        mask = torch.rand_like(ids.float()) < self.aug_params['dropout_prob']
        ids[mask & (ids != PAD)] = MASK
        enc['input_ids'] = ids

    # ----- 2. Swap neighbouring tokens -----
    def _swap_neighbor(self, enc):
        ids = enc['input_ids']
        for i in range(1, len(ids) - 1):
            if random.random() < self.aug_params['swap_prob'] and ids[i] not in (PAD, MASK):
                ids[i], ids[i + 1] = ids[i + 1].clone(), ids[i].clone()
        enc['input_ids'] = ids

    # ----- 3. Random inner padding -----
    def _random_pad(self, enc):
        ids, mask = enc['input_ids'], enc['attention_mask']
        pad_prob = self.aug_params['pad_prob']
        new_ids, new_mask = [], []
        for tok, m in zip(ids, mask):
            if m.item() == 0:  # 已到 PAD 区
                break
            new_ids.append(tok.item())
            new_mask.append(1)
            if random.random() < pad_prob and len(new_ids) < self.max_length - 1:
                new_ids.append(PAD)
                new_mask.append(0)
        # 截断 / 末尾补 PAD
        new_ids = (new_ids + [PAD] * self.max_length)[:self.max_length]
        new_mask = (new_mask + [0] * self.max_length)[:self.max_length]
        enc['input_ids'] = torch.tensor(new_ids, dtype=torch.long)
        enc['attention_mask'] = torch.tensor(new_mask, dtype=torch.long)

    # ----- 4. CutMix (同标签) -----
    def _cutmix(self, enc, lab):
        # 随机再挑一条同标签样本
        same_idxs = [i for i, y in enumerate(self.labels) if y == lab]
        other = {k: v.clone() for k, v in self.encodings[random.choice(same_idxs)].items()}
        lam = random.uniform(self.aug_params['cutmix_min'],
                             self.aug_params['cutmix_max'])
        cut_point = int(lam * self.max_length)
        # 前半取当前，后半取 other
        enc['input_ids'][cut_point:] = other['input_ids'][cut_point:]
        enc['attention_mask'][cut_point:] = other['attention_mask'][cut_point:]


# ========== 用 法 ========== #
with open('./data/train-claims.json') as f:
    train_claims = json.load(f)
with open('./data/evidence.json') as f:
    evidence_dict = json.load(f)

train_dataset = ClaimEvidenceDataset(
    train_claims,
    evidence_dict,
    tokenizer,
    max_length=512,
    balance=True,                # ✨ 打开类别均衡
    target_ratio=1.0,            # 把所有类补到与最大类一样多
    augmenters=['dropout', 'swap', 'pad', 'cutmix'],
    aug_params=dict(dropout_prob=0.15, swap_prob=0.1,
                    pad_prob=0.05, cutmix_min=0.4, cutmix_max=0.6)
)

print("最终样本量:", len(train_dataset))
print("类别计数:", Counter(train_dataset.labels))


最终样本量: 7720
类别计数: Counter({3: 1930, 1: 1930, 0: 1930, 2: 1930})


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [6]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
train = True
dev_train_included = True
if (dev_train_included):
    model_path = "./model/final_bert_classifier"
else:
    model_path = "./model/my_bert_classifier"
if (train):
    # 加载数据
    with open('./data/train-claims.json', 'r') as f:
        train_claims = json.load(f)
    with open('./data/evidence.json', 'r') as f:
        evidence_dict = json.load(f)
    if (dev_train_included):
        with open('./data/dev-claims.json','r') as f:
            dev_claims = json.load(f)

        all_claims = {**train_claims, **dev_claims}   # dev 覆盖 train 的同名 key；反之亦可

        # ---------- 3. 构造数据集 ----------
        train_dataset = ClaimEvidenceDataset(
            claims=all_claims,
            evidence_dict=evidence_dict,
            tokenizer=tokenizer,
            max_length=512,
            balance=True,             # 若要做类别均衡 / 增广就保持 True
            target_ratio=1.0,
        )
    else:
        train_dataset = train_dataset = ClaimEvidenceDataset(
            claims=train_claims,
            evidence_dict=evidence_dict,
            tokenizer=tokenizer,
            max_length=512,
            balance=True,             # 若要做类别均衡 / 增广就保持 True
            target_ratio=1.0,
        )
    # 加载 BERT 模型
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

    # 设置训练参数
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=16,
        num_train_epochs=3,
        eval_strategy="no",
        save_strategy="no",
        logging_strategy="no",   
        logging_steps=50,
    )

    # 初始化 Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer
    )

    # 开始训练
    trainer.train()

    # 保存模型

    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\DELL\AppData\Local\Temp\ipykernel_14776\4187663085.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


KeyboardInterrupt: 

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [3]:
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import torch.nn.functional as F

# 1. 加载模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = BertTokenizer.from_pretrained(model_path)
model.eval()

# 2. 加载你的 retrieval 文件（包含 claim_text）
with open("test-claims-predictions.json", "r") as f:
    test_data = json.load(f)

# 3. 加载 evidence 内容
with open("./data/evidence.json", "r") as f:
    evidence_dict = json.load(f)

id2label = {
    0: "SUPPORTS",
    1: "REFUTES",
    2: "NOT_ENOUGH_INFO",
    3: "DISPUTED"
}

# 4. 定义平均概率融合预测函数
def predict_label_average(claim_text, evidence_ids):
    probs_list = []

    for eid in evidence_ids:
        if eid not in evidence_dict:
            continue
        ev_text = evidence_dict[eid]

        inputs = tokenizer(
            claim_text,
            ev_text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1)
            probs_list.append(probs.cpu().numpy()[0])

    if not probs_list:
        return "NOT_ENOUGH_INFO"  # fallback

    avg_probs = np.mean(probs_list, axis=0)
    pred_idx = int(np.argmax(avg_probs))
    return id2label[pred_idx]

# 5. 预测所有 test claim
final_predictions = {}

for cid, entry in tqdm(test_data.items(), desc="Classifying"):
    claim_text = entry["claim_text"]
    evidence_ids = entry["evidence"]

    label = predict_label_average(claim_text, evidence_ids)

    final_predictions[cid] = {
        "evidences": evidence_ids,
        "claim_label": label,
        "claim_text": claim_text
    }

# 6. 保存
with open("test-claims-final-predictions.json", "w") as f:
    json.dump(final_predictions, f, indent=2)

print("✅ Done: saved to test-claims-final-predictions.json")


Classifying: 100%|██████████| 153/153 [00:15<00:00, 10.11it/s]

✅ Done: saved to test-claims-final-predictions.json


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*